In [1]:
# Plot optics, rays, celestial coords, and various official Rubin coordinate systems in 3D.
# Some notes: 
#   - "telescope parked position" is
#     • altitude = 90 degrees
#     • azimuth = 90 degrees  (i.e., telescope would point East if tipped off zenith)
#     • rotation = 0 degrees  
#   - batoid.globalCoordSys = ipvolume coordinate system = Observatory Mount Coordinate System (MCS)

In [2]:
import ipywidgets
import batoid
import numpy as np
import ipyvolume as ipv
import matplotlib.pyplot as plt

In [3]:
fiducial_telescope = batoid.Optic.fromYaml("LSST_r.yaml")
dist = 15000  # "distance" in m to Celestial sphere for plotting purposes

In [4]:
def xfan(telescope, theta_x, theta_y, nx, **kwargs):
    return batoid.RayVector.asGrid(telescope, wavelength=625e-9, theta_x=theta_x, theta_y=theta_y, nx=nx, ny=1, **kwargs)
def yfan(telescope, theta_x, theta_y, ny, **kwargs):
    return batoid.RayVector.asGrid(telescope, wavelength=625e-9, theta_x=theta_x, theta_y=theta_y, nx=1, ny=ny, **kwargs)
def xyfan(telescope, theta_x, theta_y, n, **kwargs):
    rays1 = xfan(telescope, theta_x, theta_y, n, **kwargs)
    rays2 = yfan(telescope, theta_x, theta_y, n, **kwargs)
    return batoid.concatenateRayVectors([rays1, rays2])

In [5]:
def plotlineipv(ipv, ctf, x, y, z):
    x, y, z = np.broadcast_arrays(x, y, z)
    x, y, z = ctf.applyForwardArray(x, y, z)
    ipv.plot(x, y, z, c='r')

In [6]:
def draw3dFP(ipv, coordSys):
    # Just has to be approximately right...
    raftsize = 0.31527*2/5
    ctf = batoid.CoordTransform(coordSys, batoid.globalCoordSys)
    
    # top hline
    plotlineipv(ipv, ctf, np.array([-1.5, 1.5])*raftsize, 2.5*raftsize, 0)
    # middle hlines
    plotlineipv(ipv, ctf, np.array([-2.5, 2.5])*raftsize, 1.5*raftsize, 0)
    plotlineipv(ipv, ctf, np.array([-2.5, 2.5])*raftsize, 0.5*raftsize, 0)
    plotlineipv(ipv, ctf, np.array([-2.5, 2.5])*raftsize, -0.5*raftsize, 0)
    plotlineipv(ipv, ctf, np.array([-2.5, 2.5])*raftsize, -1.5*raftsize, 0)
    # bottom hline
    plotlineipv(ipv, ctf, np.array([-1.5, 1.5])*raftsize, -2.5*raftsize, 0)
    # left vline
    plotlineipv(ipv, ctf, -2.5*raftsize, np.array([-1.5, 1.5])*raftsize, 0)
    # middle vlines
    plotlineipv(ipv, ctf, -1.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    plotlineipv(ipv, ctf, -0.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    plotlineipv(ipv, ctf, 0.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    plotlineipv(ipv, ctf, 1.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    # right vline
    plotlineipv(ipv, ctf, 2.5*raftsize, np.array([-1.5, 1.5])*raftsize, 0)

In [7]:
def drawSkyFP(ipv, alt, az, rot):
    ctf = batoid.CoordTransform(
        batoid.globalCoordSys,
        batoid.CoordSys(
            (0, 0, 0),
            batoid.RotX(-rot)@batoid.RotY(alt)@batoid.RotZ(az)
        )
    )
    f = dist*np.deg2rad(0.7)

    # top hline
    plotlineipv(ipv, ctf, dist, np.array([-1.5, 1.5])*f, 2.5*f*np.ones(2))
    # middle hlines
    plotlineipv(ipv, ctf, dist, np.array([-2.5, 2.5])*f, 1.5*f*np.ones(2))
    plotlineipv(ipv, ctf, dist, np.array([-2.5, 2.5])*f, 0.5*f*np.ones(2))
    plotlineipv(ipv, ctf, dist, np.array([-2.5, 2.5])*f, -0.5*f*np.ones(2))
    plotlineipv(ipv, ctf, dist, np.array([-2.5, 2.5])*f, -1.5*f*np.ones(2))    
    # bottom hline
    plotlineipv(ipv, ctf, dist, np.array([-1.5, 1.5])*f, -2.5*f*np.ones(2))
    # left vline
    plotlineipv(ipv, ctf, dist, -2.5*f*np.ones(2), np.array([-1.5, 1.5])*f)
    # middle vlines
    plotlineipv(ipv, ctf, dist, -1.5*f*np.ones(2), np.array([-2.5, 2.5])*f)
    plotlineipv(ipv, ctf, dist, -0.5*f*np.ones(2), np.array([-2.5, 2.5])*f)
    plotlineipv(ipv, ctf, dist, 0.5*f*np.ones(2), np.array([-2.5, 2.5])*f)
    plotlineipv(ipv, ctf, dist, 1.5*f*np.ones(2), np.array([-2.5, 2.5])*f)
    # right vline
    plotlineipv(ipv, ctf, dist, 2.5*f*np.ones(2), np.array([-1.5, 1.5])*f)

In [8]:
def plotlineax(ax, ctf, x, y):
    x, y, z = np.broadcast_arrays(x, y, 0)
    x, y, _ = ctf.applyForwardArray(x, y, z)
    ax.plot(x, y, c='r')

In [9]:
def draw2dFP(ax, coordSys):
    # Just has to be approximately right...
    raftsize = 0.31527*2/5
    ctf = batoid.CoordTransform(coordSys, batoid.globalCoordSys)
    # top hline
    plotlineax(ax, ctf, np.array([-1.5, 1.5])*raftsize, 2.5*raftsize)
    # middle hlines
    plotlineax(ax, ctf, np.array([-2.5, 2.5])*raftsize, 1.5*raftsize)
    plotlineax(ax, ctf, np.array([-2.5, 2.5])*raftsize, 0.5*raftsize)
    plotlineax(ax, ctf, np.array([-2.5, 2.5])*raftsize, -0.5*raftsize)
    plotlineax(ax, ctf, np.array([-2.5, 2.5])*raftsize, -1.5*raftsize)
    # bottom hline
    plotlineax(ax, ctf, np.array([-1.5, 1.5])*raftsize, -2.5*raftsize)
    # left vline
    plotlineax(ax, ctf, -2.5*raftsize, np.array([-1.5, 1.5])*raftsize)
    # middle vlines
    plotlineax(ax, ctf, -1.5*raftsize, np.array([-2.5, 2.5])*raftsize)
    plotlineax(ax, ctf, -0.5*raftsize, np.array([-2.5, 2.5])*raftsize)
    plotlineax(ax, ctf, 0.5*raftsize, np.array([-2.5, 2.5])*raftsize)
    plotlineax(ax, ctf, 1.5*raftsize, np.array([-2.5, 2.5])*raftsize)
    # right vline
    plotlineax(ax, ctf, 2.5*raftsize, np.array([-1.5, 1.5])*raftsize)    

In [10]:
def drawAzimuthRing(ipv):
    th = np.linspace(0, 2*np.pi, 100)
    x_, y_ = np.cos(th), np.sin(th)
    for d in [-0.1, 0.1]:
        for r in [4.5, 5.0]:
            x = x_*r
            y = y_*r
            z = d
            ipv.plot(x, y, z, color='white')

In [11]:
def drawElevationBearings(ipv, az):
    th = np.linspace(0, 2*np.pi, 100)
    x_, y_ = np.cos(th), np.sin(th)
    for d in [4.4, 4.5]:
        for r in [1.5, 1.75]:
            x = -d
            y = x_*r
            z = y_*r            
            z += 5.425  # height of elevation axis above azimuth ring
            x, y = np.cos(np.pi/2-az)*x-np.sin(np.pi/2-az)*y, np.sin(np.pi/2-az)*x+np.cos(np.pi/2-az)*y
            
            ipv.plot(x, y, z, color='white')
            ipv.plot(-x, -y, z, color='white')

In [12]:
def drawAltAzGrid(ipv):
    th = np.linspace(0, 2*np.pi, 100)
    for alt in np.arange(0, 81, 10):
        x = np.cos(th)*np.cos(np.deg2rad(alt))
        y = np.sin(th)*np.cos(np.deg2rad(alt))
        z = np.full_like(th, np.sin(np.deg2rad(alt)))
        ipv.plot(x*dist, y*dist, z*dist, color='green')
    th = np.linspace(0, np.pi/2, 50)
    for az in np.linspace(0, 2*np.pi, 12, endpoint=False):
        x = np.cos(az)*np.cos(th)
        y = np.sin(az)*np.cos(th)
        z = np.sin(th)
        if az == 0:
            ipv.plot(x*dist, y*dist, z*dist, color='red')
        else:
            ipv.plot(x*dist, y*dist, z*dist, color='green')

In [13]:
def filter_xyz_horizon(x, y, z):
    w = z>0
    if np.any(w) and not np.all(w):
        indices = np.where(np.diff(w.astype(int))==1)[0]
        if len(indices) != 1:
            idx = 0
        else:
            idx = indices[0]+1
        w = np.roll(w, -idx)
        x = np.roll(x, -idx)
        y = np.roll(y, -idx)
        z = np.roll(z, -idx)
    return w, x, y, z

In [14]:
def drawEqGrid(ipv, lat):
    # Need to rotate around +y axis by 90-lat
    ctf = batoid.CoordTransform(
        batoid.globalCoordSys,
        batoid.CoordSys(
            (0,0,0), 
            batoid.RotY(-(np.pi/2-lat))
        )
    )
    th = np.linspace(0, 2*np.pi, 100)
    for alt in np.arange(-90, 91, 10):
        x = np.cos(th)*np.cos(np.deg2rad(alt))
        y = np.sin(th)*np.cos(np.deg2rad(alt))
        z = np.full_like(th, np.sin(np.deg2rad(alt)))
        x, y, z = ctf.applyForwardArray(x, y, z)
        w, x, y, z = filter_xyz_horizon(x, y, z)
        if np.any(w):
            ipv.plot(x[w]*dist, y[w]*dist, z[w]*dist, color='blue')
    for az in np.linspace(0, 2*np.pi, 12, endpoint=False):
        x = np.cos(az)*np.cos(th)
        y = np.sin(az)*np.cos(th)
        z = np.sin(th)
        x, y, z = ctf.applyForwardArray(x, y, z)
        w, x, y, z = filter_xyz_horizon(x, y, z)
        if np.any(w):
            ipv.plot(x[w]*dist, y[w]*dist, z[w]*dist, color='blue')

In [15]:
def getAzCS(az):
    """Return Azimuth Coordinate System (AzCS).  (LTS-136)

    AzCS is defined by:
        • +z towards zenith
        • Telescope at horizon points towards +y
        • +x completes RHR
        • Origin at center of azimuth ring.
        
    Parameters
    ----------
    az : float
        Azimuth in radians
    
    Returns
    -------
    batoid.CoordSys
    """
    return batoid.CoordSys(
        origin = [0, 0, 0],
        rot = batoid.RotZ(np.pi/2-az)
    )

In [16]:
def getOpCS(alt, az):
    """Return Optical Coordinate System (OpCS).  (LTS-136)

    OpCS is defined by:
        • +x same as +x in AzCS
        • +z along boresight, towards sky
        • +y completes RHR        
        • Origin at M1 vertex
        
    Parameters
    ----------
    alt : float
        Altitude in radians
    az : float
        Azimuth in radians
    
    Returns
    -------
    batoid.CoordSys
    """
    # Must be a better way to do this than using the complete telescope,
    # but this works for now.
    telescope = fiducial_telescope
    telescope = telescope.withGlobalShift([0, 0, 3.53])  # Height of M1 vertex above azimuth ring
    telescope = telescope.withLocalRotation(batoid.RotZ(np.pi/2-az))
    telescope = telescope.withLocalRotation(batoid.RotX(np.pi/2-alt), rotOrigin=[0, 0, 5.425], coordSys=batoid.globalCoordSys)
    return telescope['M1'].coordSys

In [17]:
def getMCS():
    """Return Observatory Mount Coordinate System (MCS).  (LCA-280)
    
    MCS is the same as ipv axes, or batoid.globalCoordSys
    
    Defined by:
        • +Z towards zenith
        • +X towards North
        • +Y from RHR (towards West)
        • Origin at center of azimuth ring.
        
    Returns
    -------
    batoid.CoordSys
    """
    return batoid.globalCoordSys

In [18]:
def getTCS(alt, az):
    """Return Telescope Coordinate System (TCS).  (LCA-280)
        
    Defined by:
        • +Z towards sky along optic axis.  Follows telescope.
        • +X towards North when telescope parked.  Follows telescope.
        • +Y from RHR (towards West when parked).  Follows telescope.
        • Origin at M1 vertex

    Parameters
    ----------
    alt : float
        Altitude in radians
    az : float
        Azimuth in radians

    Returns
    -------
    batoid.CoordSys
    """
    return getOpCS(alt, az)

In [19]:
def getZCS(alt, az):
    """Return Zemax Coordinate System (https://sitcomtn-003.lsst.io/)
    
    Defined by:
        • +Z from sky towards M1M3
        • +X same as 
    
    """
    cs = getOpCS(alt, az).copy()
    cs.rot[:, 0] *= -1
    cs.rot[:, 2] *= -1
    return cs

In [20]:
def drawCoordSys(ipv, coordSys, length=2):
    p0 = coordSys.origin
    px = p0 + coordSys.rot @ (np.array([1, 0, 0]) * length)
    py = p0 + coordSys.rot @ (np.array([0, 1, 0]) * length)
    pz = p0 + coordSys.rot @ (np.array([0, 0, 1]) * length)
    csx = ipv.plot(*np.vstack([p0, px]).T, color='blue')
    csy = ipv.plot(*np.vstack([p0, py]).T, color='red')
    csz = ipv.plot(*np.vstack([p0, pz]).T, color='white')
    return csx, csy, csz

In [21]:
def ten(s):
    ss = s.split()
    h = ss[0]
    m = ss[1]
    s = ss[2]
    return float(h) + float(m)/60 + float(s)/3600

In [22]:
# del drawConstellations
def drawConstellations(ipv, lat):
    ctf = batoid.CoordTransform(
        batoid.globalCoordSys,
        batoid.CoordSys(
            (0,0,0), 
            batoid.RotY(-(np.pi/2-lat))
        )
    )

    if not hasattr(drawConstellations, 'data'):
        from astroquery.simbad import Simbad
        Simbad.add_votable_fields('typed_id')
        out = Simbad.query_objects(["HIP 67301", "HIP 65378"])
        HIPset = set()
        with open("constellationship.fab") as f:
            lines = f.readlines()
        for line in lines:
            HIPset.update([int(s) for s in line.split()[2:]])
        HIPlist = list(HIPset)
        data = Simbad.query_objects(
            [f"HIP {s}" for s in HIPlist]
        )
        data['HIPID'] = HIPlist
        drawConstellations.data = data
        drawConstellations.lines = lines
    data = drawConstellations.data
    lines = drawConstellations.lines
    for line in lines:
        endpoints = iter(line.split()[2:])        
        for first in endpoints:
            second = next(endpoints)
            firstrow = data[np.nonzero(data['HIPID'] == int(first))]
            secondrow = data[np.nonzero(data['HIPID'] == int(second))]
            ra0 = np.deg2rad(15*ten(firstrow['RA'][0]))
            dec0 = np.deg2rad(ten(firstrow['DEC'][0]))
            ra1 = np.deg2rad(15*ten(secondrow['RA'][0]))
            dec1 = np.deg2rad(ten(secondrow['DEC'][0]))
            x0 = np.cos(ra0)*np.cos(dec0)
            y0 = np.sin(ra0)*np.cos(dec0)
            z0 = np.sin(dec0)
            x1 = np.cos(ra1)*np.cos(dec1)
            y1 = np.sin(ra1)*np.cos(dec1)
            z1 = np.sin(dec1)
            
            x = np.array([x0, x1])*dist
            y = np.array([y0, y1])*dist
            z = np.array([z0, z1])*dist
            x, y, z = ctf.applyForwardArray(x, y, z)
            if np.any(z<0):
                continue

            ipv.plot(x, y, z, color='white')

In [23]:
def show_coord_sys(
    rot, alt, az, thx, thy
):
    lat = np.deg2rad(-30.2446)
    telescope = fiducial_telescope
    telescope = telescope.withGlobalShift([0, 0, 3.53])  # Height of M1 vertex above azimuth ring
    telescope = telescope.withLocallyRotatedOptic("LSSTCamera", batoid.RotZ(rot))
    telescope = telescope.withLocalRotation(batoid.RotZ(np.pi/2-az))
    telescope = telescope.withLocalRotation(batoid.RotX(np.pi/2-alt), rotOrigin=[0, 0, 5.425], coordSys=batoid.globalCoordSys)

    rays = xyfan(telescope, thx, thy, 30)
    traceFull = telescope.traceFull(rays)

    ipvfig = ipv.figure(width=600, height=450)
    
    drawAzimuthRing(ipv)
    drawElevationBearings(ipv, az)    
    telescope.draw3d(ipv, color='white')

    batoid.drawTrace3d(ipv, traceFull, color='blue')

    # extent rays all the way back to the Celestial sphere
    w = ~traceFull['M1']['out'].vignetted
    rays = rays.toCoordSys(batoid.globalCoordSys)
    p0 = rays[w].r
    p1 = rays[w].positionAtTime(-dist)
    for _p0, _p1 in zip(p0, p1):
        ipv.plot(*np.vstack([_p0, _p1]).T, color='blue')
    
    draw3dFP(ipv, telescope['Detector'].coordSys)
    drawSkyFP(ipv, alt, az, rot)
    
    drawAltAzGrid(ipv)
    drawEqGrid(ipv, lat)

    AzCS = getAzCS(az)
    AzCS = batoid.CoordSys(
        origin = [0, 0, 0],
        rot = batoid.RotZ(np.pi/2-az)
    )
    AzCSWidgets = drawCoordSys(ipv, AzCS)
    OpCS = getOpCS(alt, az)
    OpCSWidgets = drawCoordSys(ipv, OpCS)                         
    MCS = getMCS()
    MCSWidgets = drawCoordSys(ipv, MCS)
    TCS = getTCS(alt, az)
    TCSWidgets = drawCoordSys(ipv, TCS)
    CCSWidgets = drawCoordSys(ipv, telescope['LSSTCamera'].coordSys)            
    ZCS = getZCS(alt, az)
    ZCSWidgets = drawCoordSys(ipv, ZCS)
            
    drawConstellations(ipv, lat)
            
    gcc = ipv.gcc()
    ipv.xlim(-8, 8)
    ipv.ylim(-8, 8)
    ipv.zlim(0, 16)
    ipv.style.axes_off()
    ipv.style.box_off()
    ipv.style.set_style_dark()
    ipvfig.camera.position = (-1.5254697593501128, -1.2928322819658806, 0.03907305996705254)
    ipvfig.camera.rotation = (-1.2237988550857888, -1.3392070664612021, -2.763432436269493, 'XYZ')

    doAzCSBox = ipywidgets.Checkbox(True, description="AzCS")
    doOpCSBox = ipywidgets.Checkbox(True, description="OpCS")
    doMCSBox = ipywidgets.Checkbox(True, description="MCS")
    doTCSBox = ipywidgets.Checkbox(True, description="TCS")
    doCCSBox = ipywidgets.Checkbox(True, description="CCS")
    doZCSBox = ipywidgets.Checkbox(True, description="ZCS")
    
    def showCS(value, CS):
        CS[0].visible = not CS[0].visible
        CS[1].visible = not CS[1].visible
        CS[2].visible = not CS[2].visible
    
    doAzCSBox.observe(lambda change: showCS(change, AzCSWidgets), 'value')
    doOpCSBox.observe(lambda change: showCS(change, OpCSWidgets), 'value')
    doMCSBox.observe(lambda change: showCS(change, MCSWidgets), 'value')
    doTCSBox.observe(lambda change: showCS(change, TCSWidgets), 'value')
    doCCSBox.observe(lambda change: showCS(change, CCSWidgets), 'value')
    doZCSBox.observe(lambda change: showCS(change, ZCSWidgets), 'value')
    
    return ipywidgets.HBox([gcc, ipywidgets.VBox([doAzCSBox, doOpCSBox, doMCSBox, doTCSBox, doCCSBox, doZCSBox])])

In [24]:
# rot = 0.7485759992183523 
rot = 0.1
alt = 0.8010022880203016 
az = 1.8888812762063498 

thx = np.deg2rad(0.0)
thy = np.deg2rad(1.75)
show_coord_sys(rot, alt, az, thx, thy)